In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")
NESSIE_URI = os.getenv("NESSIE_URI")

MASTER = "spark://spark-master:7077"

jar_packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
    "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.99.0",
    "org.apache.iceberg:iceberg-aws-bundle:1.6.1"
  ]

spark_extensions = [
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
]
conf = (
    pyspark.SparkConf()
    .setAppName("Silver Iceberg")
    .set("spark.master", MASTER)
    .set("spark.jars.packages", ','.join(jar_packages))
    .set("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    .set("spark.executor.memory", "2g")
    .set("spark.executor.cores", "2")
    .set("spark.executor.instances", "1")
    .set("spark.driver.memory", "2g")
    .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .set("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .set("spark.sql.catalog.nessie.s3.endpoint", S3_ENDPOINT)
    .set("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
    .set("spark.sql.catalog.nessie.ref", "main")
    .set("spark.sql.catalog.nessie.authentication.type", "NONE")
    .set("spark.sql.catalog.nessie.cache-enabled", "false")
    .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
    .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
    .set("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-10371fbe-5845-444a-86d5-21299f02aff3;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.99.0 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.6.1 in central
:: resolution report :: resolve 723ms :: artifacts dl 30ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.6.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.99.0 from cen

In [3]:
# leitura da tabela iceberg da camada silver e criação de uma tabela temporaria
spark.read.format("iceberg").load("nessie.silver.tabela_enem_silver").createOrReplaceTempView("tabela_enem_silver")

In [4]:
spark.sql("""
CREATE NAMESPACE IF NOT EXISTS nessie.gold
""")

DataFrame[]

**CRIAÇÃO DA TABELA DE DIM.APLIACAO_PROVA NA CAMADA GOLD**

In [24]:
spark.sql("""
    CREATE TABLE nessie.gold.dim_aplicacao_prova
    USING iceberg
    LOCATION 's3a://gold/dim_aplicacao_prova/'
    AS SELECT 
        ID AS ID_APLICACAO_PROVA,
        CO_MUNICIPIO_PROVA,
        NO_MUNICIPIO_PROVA,
        CO_UF_PROVA,
        SG_UF_PROVA
      FROM tabela_enem_silver LIMIT 0;
    """)

spark.sql("""
    INSERT INTO nessie.gold.dim_aplicacao_prova
    SELECT 
      ID AS ID_APLICACAO_PROVA,
      CO_MUNICIPIO_PROVA,
      NO_MUNICIPIO_PROVA,
      CO_UF_PROVA,
      SG_UF_PROVA
    FROM tabela_enem_silver;
    """)

DataFrame[]

In [26]:
spark.sql("""SELECT * FROM nessie.gold.dim_aplicacao_prova""").show()

+------------------+------------------+------------------+-----------+-----------+
|ID_APLICACAO_PROVA|CO_MUNICIPIO_PROVA|NO_MUNICIPIO_PROVA|CO_UF_PROVA|SG_UF_PROVA|
+------------------+------------------+------------------+-----------+-----------+
|           4583466|           2408102|             Natal|         24|         RN|
|           4583467|           2407203|             Macau|         24|         RN|
|           4583468|           2408102|             Natal|         24|         RN|
|           4583469|           2408102|             Natal|         24|         RN|
|           4583470|           2402006|             Caic�|         24|         RN|
|           4583471|           2404200|         Goianinha|         24|         RN|
|           4583472|           2402006|             Caic�|         24|         RN|
|           4583473|           2408102|             Natal|         24|         RN|
|           4583474|           2406106|          Jucurutu|         24|         RN|
|   

**CRIAÇÃO DA TABELA DE DIM.ESCOLA NA CAMADA GOLD**

In [28]:
spark.sql("""
    CREATE TABLE nessie.gold.dim_escola
    USING iceberg
    LOCATION 's3a://gold/dim_escola/'
    AS SELECT 
        ID AS ID_ESCOLA,
        CO_MUNICIPIO_ESC,
        NO_MUNICIPIO_ESC,
        CO_UF_ESC,
        SG_UF_ESC,
        TP_DEPENDENCIA_ADM_ESC,
        TP_LOCALIZACAO_ESC,
        TP_SIT_FUNC_ESC
      FROM tabela_enem_silver LIMIT 0;
    """)

spark.sql("""
    INSERT INTO nessie.gold.dim_escola
    SELECT 
      ID AS ID_ESCOLA,
      CO_MUNICIPIO_ESC,
      NO_MUNICIPIO_ESC,
      CO_UF_ESC,
      SG_UF_ESC,
    CASE
      WHEN TP_DEPENDENCIA_ADM_ESC = '1' THEN 'Federal'
      WHEN TP_DEPENDENCIA_ADM_ESC = '2' THEN 'Estadual'
      WHEN TP_DEPENDENCIA_ADM_ESC = '3' THEN 'Municipal'
      WHEN TP_DEPENDENCIA_ADM_ESC = '4' THEN 'Privada'
      ELSE TP_DEPENDENCIA_ADM_ESC
    END AS TP_DEPENDENCIA_ADM_ESC,
    CASE
      WHEN TP_LOCALIZACAO_ESC = '1' THEN 'Urbana'
      WHEN TP_LOCALIZACAO_ESC = '2' THEN 'Rural'
      ELSE TP_LOCALIZACAO_ESC
    END AS TP_LOCALIZACAO_ESC,
    CASE
      WHEN TP_SIT_FUNC_ESC = '1' THEN 'Em atividade'
      WHEN TP_SIT_FUNC_ESC = '2' THEN 'Paralisada'
      WHEN TP_SIT_FUNC_ESC = '3' THEN 'Extinta'
      ELSE TP_SIT_FUNC_ESC
    END AS TP_SIT_FUNC_ESC
    FROM tabela_enem_silver;
    """)

DataFrame[]

In [33]:
spark.sql("""SELECT * FROM nessie.gold.dim_escola WHERE CO_MUNICIPIO_ESC != 0""").show()

+---------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+
|ID_ESCOLA|CO_MUNICIPIO_ESC|    NO_MUNICIPIO_ESC|CO_UF_ESC|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|TP_SIT_FUNC_ESC|
+---------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+
|  4583484|         2408102|               Natal|       24|       RN|              Estadual|            Urbana|   Em atividade|
|  4583488|         2408102|               Natal|       24|       RN|              Estadual|            Urbana|   Em atividade|
|  4583490|         2412005|S�o Gon�alo do Am...|       24|       RN|              Estadual|            Urbana|   Em atividade|
|  4583507|         2413003|         S�o Vicente|       24|       RN|              Estadual|            Urbana|   Em atividade|
|  4583508|         2403251|          Parnamirim|       24|       RN|              Estadual|            

**CRIAÇÃO DA TABELA DE DIM.PARTICIPANTE NA CAMADA GOLD**

In [34]:
spark.sql("""
    CREATE TABLE nessie.gold.dim_participante
    USING iceberg
    LOCATION 's3a://gold/dim_participante/'
    AS SELECT 
        ID AS ID_PARTICIPANTE,
        NU_INSCRICAO,
        NU_ANO,
        TP_FAIXA_ETARIA,
        TP_SEXO,
        TP_ESTADO_CIVIL,
        TP_COR_RACA,
        TP_NACIONALIDADE,
        TP_ST_CONCLUSAO,
        TP_ANO_CONCLUIU,
        TP_ESCOLA,
        TP_ENSINO,
        IN_TREINEIRO
      FROM tabela_enem_silver LIMIT 0;
    """)

spark.sql("""
    INSERT INTO nessie.gold.dim_participante
    SELECT 
      ID AS ID_PARTICIPANTE,
      NU_INSCRICAO,
      NU_ANO,
      CASE
          WHEN TP_FAIXA_ETARIA = '1' THEN 'Menor de 17 anos'
          WHEN TP_FAIXA_ETARIA = '2' THEN '17 anos'
          WHEN TP_FAIXA_ETARIA = '3' THEN '18 anos'
          WHEN TP_FAIXA_ETARIA = '4' THEN '19 anos'
          WHEN TP_FAIXA_ETARIA = '5' THEN '20 anos'
          WHEN TP_FAIXA_ETARIA = '6' THEN '21 anos'
          WHEN TP_FAIXA_ETARIA = '7' THEN '22 anos'
          WHEN TP_FAIXA_ETARIA = '8' THEN '23 anos'
          WHEN TP_FAIXA_ETARIA = '9' THEN '24 anos'
          WHEN TP_FAIXA_ETARIA = '10' THEN '25 anos'
          WHEN TP_FAIXA_ETARIA = '11' THEN 'Entre 26 a 30 anos'
          WHEN TP_FAIXA_ETARIA = '12' THEN 'Entre 31 a 35 anos'
          WHEN TP_FAIXA_ETARIA = '13' THEN 'Entre 36 a 40 anos'
          WHEN TP_FAIXA_ETARIA = '14' THEN 'Entre 41 a 45 anos'
          WHEN TP_FAIXA_ETARIA = '15' THEN 'Entre 46 a 50 anos'
          WHEN TP_FAIXA_ETARIA = '16' THEN 'Entre 51 a 55 anos'
          WHEN TP_FAIXA_ETARIA = '17' THEN 'Entre 56 a 60 anos'
          WHEN TP_FAIXA_ETARIA = '18' THEN 'Entre 61 a 65 anos'
          WHEN TP_FAIXA_ETARIA = '19' THEN 'Entre 66 a 70 anos'
          WHEN TP_FAIXA_ETARIA = '20' THEN 'Maior de 70 anos'
          ELSE TP_FAIXA_ETARIA
      END AS TP_FAIXA_ETARIA,
      CASE
          WHEN TP_SEXO = 'F' THEN 'Feminino'
          WHEN TP_SEXO = 'M' THEN 'Masculino'
          ELSE 'Não encontrado'
      END AS TP_SEXO,
      CASE
          WHEN TP_ESTADO_CIVIL = '0' THEN 'Não informado'
          WHEN TP_ESTADO_CIVIL = '1' THEN 'Solteiro(a)'
          WHEN TP_ESTADO_CIVIL = '2' THEN 'Casado(a) Mora com companheiro(a)'
          WHEN TP_ESTADO_CIVIL = '3' THEN 'Divorciado(a)/Desquitado(a)/Separado(a)'
          WHEN TP_ESTADO_CIVIL = '4' THEN 'Viúvo(a)'
          ELSE TP_ESTADO_CIVIL
      END AS TP_ESTADO_CIVIL,
      CASE
          WHEN TP_COR_RACA = '0' THEN 'Não declarado'
          WHEN TP_COR_RACA = '1' THEN 'Branca'
          WHEN TP_COR_RACA = '2' THEN 'Preta'
          WHEN TP_COR_RACA = '3' THEN 'Parda'
          WHEN TP_COR_RACA = '4' THEN 'Amarela'
          WHEN TP_COR_RACA = '5' THEN 'Indígena'
          ELSE TP_COR_RACA
      END AS TP_COR_RACA,
      CASE
          WHEN TP_NACIONALIDADE = '0' THEN 'Não informado'
          WHEN TP_NACIONALIDADE = '1' THEN 'Brasileiro(a)'
          WHEN TP_NACIONALIDADE = '2' THEN 'Brasileiro(a) Naturalizado(a)'
          WHEN TP_NACIONALIDADE = '3' THEN 'Estrangeiro(a)'
          WHEN TP_NACIONALIDADE = '4' THEN 'Brasileiro(a) Nato(a), nascido(a) no exterior'
          ELSE TP_NACIONALIDADE
      END AS TP_NACIONALIDADE,
      CASE
          WHEN TP_ST_CONCLUSAO = '1' THEN 'Já concluí o Ensino Médio'
          WHEN TP_ST_CONCLUSAO = '2' THEN 'Estou cursando e concluirei o Ensino Médio em 2020'
          WHEN TP_ST_CONCLUSAO = '3' THEN 'Estou cursando e concluirei o Ensino Médio após 2020'
          WHEN TP_ST_CONCLUSAO = '4' THEN 'Não concluí e não estou cursando o Ensino Médio'
          ELSE TP_ST_CONCLUSAO
      END AS TP_ST_CONCLUSAO,
      CASE
          WHEN TP_ANO_CONCLUIU = '0' THEN 'Não informado'
          WHEN TP_ANO_CONCLUIU = '1' THEN '2019'
          WHEN TP_ANO_CONCLUIU = '2' THEN '2018'
          WHEN TP_ANO_CONCLUIU = '3' THEN '2017'
          WHEN TP_ANO_CONCLUIU = '4' THEN '2016'
          WHEN TP_ANO_CONCLUIU = '5' THEN '2015'
          WHEN TP_ANO_CONCLUIU = '6' THEN '2014'
          WHEN TP_ANO_CONCLUIU = '7' THEN '2013'
          WHEN TP_ANO_CONCLUIU = '8' THEN '2012'
          WHEN TP_ANO_CONCLUIU = '9' THEN '2011'
          WHEN TP_ANO_CONCLUIU = '10' THEN '2010'
          WHEN TP_ANO_CONCLUIU = '11' THEN '2009'
          WHEN TP_ANO_CONCLUIU = '12' THEN '2008'
          WHEN TP_ANO_CONCLUIU = '13' THEN '2007'
          WHEN TP_ANO_CONCLUIU = '14' THEN 'Antes de 2007'
          ELSE TP_ANO_CONCLUIU
      END AS TP_ANO_CONCLUIU,
      CASE
          WHEN TP_ESCOLA = '1' THEN 'Não Respondeu'
          WHEN TP_ESCOLA = '2' THEN 'Pública'
          WHEN TP_ESCOLA = '3' THEN 'Privada'
          WHEN TP_ESCOLA = '4' THEN 'Exterior'
          ELSE TP_ESCOLA
      END AS TP_ESCOLA,
      CASE
          WHEN TP_ENSINO = '1' THEN 'Ensino Regular'
          WHEN TP_ENSINO = '2' THEN 'Educação Especial - Modalidade Substitutiva'
          WHEN TP_ENSINO = '3' THEN 'Educação de Jovens e Adultos'
          ELSE TP_ENSINO
      END AS TP_ENSINO,
      CASE
          WHEN IN_TREINEIRO = '0' THEN 'Sim'
          WHEN IN_TREINEIRO = '1' THEN 'Não'
          ELSE IN_TREINEIRO
      END AS IN_TREINEIRO
    FROM tabela_enem_silver;
    """)

DataFrame[]

In [36]:
spark.sql("""SELECT * FROM nessie.gold.dim_participante""").show()

+---------------+------------+------+------------------+---------+--------------------+-----------+----------------+--------------------+---------------+-------------+--------------+------------+
|ID_PARTICIPANTE|NU_INSCRICAO|NU_ANO|   TP_FAIXA_ETARIA|  TP_SEXO|     TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|     TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|    TP_ESCOLA|     TP_ENSINO|IN_TREINEIRO|
+---------------+------------+------+------------------+---------+--------------------+-----------+----------------+--------------------+---------------+-------------+--------------+------------+
|        4583466|190005588305|  2019|Entre 26 a 30 anos| Feminino|         Solteiro(a)|     Branca|   Brasileiro(a)|Já concluí o Ensi...|           2009|Não Respondeu|Ensino Regular|         Sim|
|        4583467|190005588306|  2019|           24 anos| Feminino|         Solteiro(a)|      Preta|   Brasileiro(a)|Já concluí o Ensi...|           2014|Não Respondeu|Ensino Regular|         Sim|
|        4583468|190

**CRIAÇÃO DA TABELA DE DIM.PROVA_OBJETIVA NA CAMADA GOLD**

In [37]:
spark.sql("""
    CREATE TABLE nessie.gold.dim_prova_objetiva
    USING iceberg
    LOCATION 's3a://gold/dim_prova_objetiva/'
    AS SELECT 
        ID AS ID_PROVA_OBJETIVA,
        TP_PRESENCA_CN,
        TP_PRESENCA_CH,
        TP_PRESENCA_LC,
        TP_PRESENCA_MT,
        CO_PROVA_CN,
        CO_PROVA_CH,
        CO_PROVA_LC,
        CO_PROVA_MT,
        NU_NOTA_CN,
        NU_NOTA_CH,
        NU_NOTA_LC,
        NU_NOTA_MT,
        TX_RESPOSTAS_CN,
        TX_RESPOSTAS_CH,
        TX_RESPOSTAS_LC,
        TX_RESPOSTAS_MT,
        TP_LINGUA,
        TX_GABARITO_CN,
        TX_GABARITO_CH,
        TX_GABARITO_LC,
        TX_GABARITO_MT
      FROM tabela_enem_silver LIMIT 0;
    """)

spark.sql("""
    INSERT INTO nessie.gold.dim_prova_objetiva
    SELECT 
      ID AS ID_PROVA_OBJETIVA,
      CASE
          WHEN TP_PRESENCA_CN = '0' THEN 'Faltou à prova'
          WHEN TP_PRESENCA_CN = '1' THEN 'Presente na prova'
          WHEN TP_PRESENCA_CN = '2' THEN 'Eliminado na prova'
          ELSE TP_PRESENCA_CN
      END AS TP_PRESENCA_CN,
      CASE
          WHEN TP_PRESENCA_CH = '0' THEN 'Faltou à prova'
          WHEN TP_PRESENCA_CH = '1' THEN 'Presente na prova'
          WHEN TP_PRESENCA_CH = '2' THEN 'Eliminado na prova'
          ELSE TP_PRESENCA_CH
      END AS TP_PRESENCA_CH,
      CASE
          WHEN TP_PRESENCA_LC = '0' THEN 'Faltou à prova'
          WHEN TP_PRESENCA_LC = '1' THEN 'Presente na prova'
          WHEN TP_PRESENCA_LC = '2' THEN 'Eliminado na prova'
          ELSE TP_PRESENCA_LC
      END AS TP_PRESENCA_LC,
      CASE
          WHEN TP_PRESENCA_MT = '0' THEN 'Faltou à prova'
          WHEN TP_PRESENCA_MT = '1' THEN 'Presente na prova'
          WHEN TP_PRESENCA_MT = '2' THEN 'Eliminado na prova'
          ELSE TP_PRESENCA_MT
      END AS TP_PRESENCA_MT,
      CASE
          WHEN CO_PROVA_CN = '597' THEN 'Azul'
          WHEN CO_PROVA_CN = '598' THEN 'Amarela'
          WHEN CO_PROVA_CN = '599' THEN 'Cinza'
          WHEN CO_PROVA_CN = '600' THEN 'Rosa'
          WHEN CO_PROVA_CN = '601' THEN 'Rosa - Ampliada'
          WHEN CO_PROVA_CN = '602' THEN 'Rosa - Superampliada'
          WHEN CO_PROVA_CN = '604' THEN 'Laranja - Adaptada Ledor'
          WHEN CO_PROVA_CN = '605' THEN 'Verde - Videoprova - Libras'
          WHEN CO_PROVA_CN = '677' THEN 'Azul (Reaplicação)'
          WHEN CO_PROVA_CN = '678' THEN 'Amarela (Reaplicação)'
          WHEN CO_PROVA_CN = '679' THEN 'Cinza (Reaplicação)'
          WHEN CO_PROVA_CN = '680' THEN 'Rosa (Reaplicação)'
          WHEN CO_PROVA_CN = '684' THEN 'Laranja - Adaptada Ledor (Reaplicação)'
          WHEN CO_PROVA_CN = '699' THEN 'Azul (Digital)'
          WHEN CO_PROVA_CN = '700' THEN 'Amarela (Digital)'
          WHEN CO_PROVA_CN = '701' THEN 'Rosa (Digital)'
          WHEN CO_PROVA_CN = '702' THEN 'Cinza (Digital)'
          ELSE 'Não encontrado'
      END AS CO_PROVA_CN,
      CASE
          WHEN CO_PROVA_CH = '567' THEN 'Azul'
          WHEN CO_PROVA_CH = '568' THEN 'Amarela'
          WHEN CO_PROVA_CH = '569' THEN 'Cinza'
          WHEN CO_PROVA_CH = '570' THEN 'Rosa'
          WHEN CO_PROVA_CH = '571' THEN 'Rosa - Ampliada'
          WHEN CO_PROVA_CH = '572' THEN 'Rosa - Superampliada'
          WHEN CO_PROVA_CH = '574' THEN 'Laranja - Adaptada Ledor'
          WHEN CO_PROVA_CH = '575' THEN 'Verde - Videoprova - Libras'
          WHEN CO_PROVA_CH = '647' THEN 'Azul (Reaplicação)'
          WHEN CO_PROVA_CH = '648' THEN 'Amarela (Reaplicação)'
          WHEN CO_PROVA_CH = '649' THEN 'Cinza (Reaplicação)'
          WHEN CO_PROVA_CH = '650' THEN 'Rosa (Reaplicação)'
          WHEN CO_PROVA_CH = '654' THEN 'Laranja - Adaptada Ledor (Reaplicação)'
          WHEN CO_PROVA_CH = '687' THEN 'Azul (Digital)'
          WHEN CO_PROVA_CH = '688' THEN 'Amarela (Digital)'
          WHEN CO_PROVA_CH = '689' THEN 'Rosa (Digital)'
          WHEN CO_PROVA_CH = '690' THEN 'Cinza (Digital)'
          ELSE 'Não encontrado'
      END AS CO_PROVA_CH,
      CASE
          WHEN CO_PROVA_LC = '577' THEN 'Azul'
          WHEN CO_PROVA_LC = '578' THEN 'Amarela'
          WHEN CO_PROVA_LC = '579' THEN 'Rosa'
          WHEN CO_PROVA_LC = '580' THEN 'Branca'
          WHEN CO_PROVA_LC = '581' THEN 'Rosa - Ampliada'
          WHEN CO_PROVA_LC = '582' THEN 'Rosa - Superampliada'
          WHEN CO_PROVA_LC = '584' THEN 'Laranja - Adaptada Ledor'
          WHEN CO_PROVA_LC = '585' THEN 'Verde - Videoprova - Libras'
          WHEN CO_PROVA_LC = '657' THEN 'Azul (Reaplicação)'
          WHEN CO_PROVA_LC = '658' THEN 'Amarela (Reaplicação)'
          WHEN CO_PROVA_LC = '659' THEN 'Rosa (Reaplicação)'
          WHEN CO_PROVA_LC = '660' THEN 'Branca (Reaplicação)'
          WHEN CO_PROVA_LC = '664' THEN 'Laranja - Adaptada Ledor (Reaplicação)'
          WHEN CO_PROVA_LC = '691' THEN 'Azul (Digital)'
          WHEN CO_PROVA_LC = '692' THEN 'Amarela (Digital)'
          WHEN CO_PROVA_LC = '693' THEN 'Branca (Digital)'
          WHEN CO_PROVA_LC = '694' THEN 'Rosa (Digital)'
          ELSE 'Não encontrado'
      END AS CO_PROVA_LC,
      CASE
          WHEN CO_PROVA_MT = '587' THEN 'Azul'
          WHEN CO_PROVA_MT = '588' THEN 'Amarela'
          WHEN CO_PROVA_MT = '589' THEN 'Rosa'
          WHEN CO_PROVA_MT = '590' THEN 'Cinza'
          WHEN CO_PROVA_MT = '591' THEN 'Rosa - Ampliada'
          WHEN CO_PROVA_MT = '592' THEN 'Rosa - Superampliada'
          WHEN CO_PROVA_MT = '594' THEN 'Laranja - Adaptada Ledor'
          WHEN CO_PROVA_MT = '595' THEN 'Verde - Videoprova - Libras'
          WHEN CO_PROVA_MT = '667' THEN 'Azul (Reaplicação)'
          WHEN CO_PROVA_MT = '668' THEN 'Amarela (Reaplicação)'
          WHEN CO_PROVA_MT = '669' THEN 'Rosa (Reaplicação)'
          WHEN CO_PROVA_MT = '670' THEN 'Cinza (Reaplicação)'
          WHEN CO_PROVA_MT = '674' THEN 'Laranja - Adaptada Ledor (Reaplicação)'
          WHEN CO_PROVA_MT = '695' THEN 'Azul (Digital)'
          WHEN CO_PROVA_MT = '696' THEN 'Amarela (Digital)'
          WHEN CO_PROVA_MT = '697' THEN 'Rosa (Digital)'
          WHEN CO_PROVA_MT = '698' THEN 'Cinza (Digital)'
          ELSE 'Não encontrado'
      END AS CO_PROVA_MT,
      NU_NOTA_CN,
      NU_NOTA_CH,
      NU_NOTA_LC,
      NU_NOTA_MT,
      TX_RESPOSTAS_CN,
      TX_RESPOSTAS_CH,
      TX_RESPOSTAS_LC,
      TX_RESPOSTAS_MT,
      CASE
          WHEN TP_LINGUA = '0' THEN 'Inglês'
          WHEN TP_LINGUA = '1' THEN 'Espanhol'
          ELSE 'Não encontrado'
      END AS TP_LINGUA,
      TX_GABARITO_CN,
      TX_GABARITO_CH,
      TX_GABARITO_LC,
      TX_GABARITO_MT
    FROM tabela_enem_silver;
    """)

DataFrame[]

In [38]:
spark.sql("""SELECT * FROM nessie.gold.dim_prova_objetiva""").show()

+-----------------+-----------------+-----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|ID_PROVA_OBJETIVA|   TP_PRESENCA_CN|   TP_PRESENCA_CH|   TP_PRESENCA_LC|   TP_PRESENCA_MT|   CO_PROVA_CN|   CO_PROVA_CH|   CO_PROVA_LC|   CO_PROVA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|     TX_RESPOSTAS_CN|     TX_RESPOSTAS_CH|     TX_RESPOSTAS_LC|     TX_RESPOSTAS_MT|TP_LINGUA|      TX_GABARITO_CN|      TX_GABARITO_CH|      TX_GABARITO_LC|      TX_GABARITO_MT|
+-----------------+-----------------+-----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+--------------------+--------------------+---------------

**CRIAÇÃO DA TABELA DE DIM.Q_SE NA CAMADA GOLD**

In [40]:
spark.sql("""
    CREATE TABLE nessie.gold.dim_q_se
    USING iceberg
    LOCATION 's3a://gold/dim_q_se/'
    AS SELECT 
      ID AS ID_Q_SE,
      Q001,
      Q002,
      Q003,
      Q004,
      Q005,
      Q006,
      Q007,
      Q008,
      Q009,
      Q010,
      Q011,
      Q012,
      Q013,
      Q014,
      Q015,
      Q016,
      Q017,
      Q018,
      Q019,
      Q020,
      Q021,
      Q022,
      Q023,
      Q024,
      Q025
      FROM tabela_enem_silver LIMIT 0;
    """)

spark.sql("""
    INSERT INTO nessie.gold.dim_q_se
    SELECT 
      ID AS ID_Q_SE,
      Q001,
      Q002,
      Q003,
      Q004,
      Q005,
      Q006,
      Q007,
      Q008,
      Q009,
      Q010,
      Q011,
      Q012,
      Q013,
      Q014,
      Q015,
      Q016,
      Q017,
      Q018,
      Q019,
      Q020,
      Q021,
      Q022,
      Q023,
      Q024,
      Q025
    FROM tabela_enem_silver;
    """)

DataFrame[]

In [42]:
spark.sql("""SELECT * FROM nessie.gold.dim_q_se""").show()

+-------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|ID_Q_SE|        Q001|        Q002|        Q003|        Q004|        Q005|        Q006|        Q007|        Q008|        Q009|        Q010|        Q011|        Q012|        Q013|        Q014|        Q015|        Q016|        Q017|        Q018|        Q019|        Q020|        Q021|        Q022|        Q023|        Q024|        Q025|
+-------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+---------

**CRIAÇÃO DA TABELA DE DIM.REDACAO NA CAMADA GOLD**

In [44]:
spark.sql("""
    CREATE TABLE nessie.gold.dim_redacao
    USING iceberg
    LOCATION 's3a://gold/dim_redacao/'
    AS SELECT 
        ID AS ID_REDACAO,
        TP_STATUS_REDACAO,
        NU_NOTA_COMP1,
        NU_NOTA_COMP2,
        NU_NOTA_COMP3,
        NU_NOTA_COMP4,
        NU_NOTA_COMP5,
        NU_NOTA_REDACAO
      FROM tabela_enem_silver LIMIT 0;
    """)

spark.sql("""
    INSERT INTO nessie.gold.dim_redacao
    SELECT 
      ID AS ID_REDACAO,
      CASE
          WHEN TP_STATUS_REDACAO = '1' THEN 'Sem problemas'
          WHEN TP_STATUS_REDACAO = '2' THEN 'Anulada'
          WHEN TP_STATUS_REDACAO = '3' THEN 'Cópia Texto Motivador'
          WHEN TP_STATUS_REDACAO = '4' THEN 'Em Branco'
          WHEN TP_STATUS_REDACAO = '6' THEN 'Fuga ao tema'
          WHEN TP_STATUS_REDACAO = '7' THEN 'Não atendimento ao tipo textual'
          WHEN TP_STATUS_REDACAO = '8' THEN 'Texto insuficiente'
          WHEN TP_STATUS_REDACAO = '9' THEN 'Parte desconectada'
          ELSE TP_STATUS_REDACAO
      END AS TP_STATUS_REDACAO,
      NU_NOTA_COMP1,
      NU_NOTA_COMP2,
      NU_NOTA_COMP3,
      NU_NOTA_COMP4,
      NU_NOTA_COMP5,
      NU_NOTA_REDACAO
    FROM tabela_enem_silver;
    """)

DataFrame[]

In [46]:
spark.sql("""SELECT * FROM nessie.gold.dim_redacao""").show()

+----------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+
|ID_REDACAO|TP_STATUS_REDACAO|NU_NOTA_COMP1|NU_NOTA_COMP2|NU_NOTA_COMP3|NU_NOTA_COMP4|NU_NOTA_COMP5|NU_NOTA_REDACAO|
+----------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+
|   4583466|    Sem problemas|        140.0|         80.0|        120.0|        120.0|         60.0|          520.0|
|   4583467|    Sem problemas|        120.0|        120.0|        120.0|        100.0|        180.0|          640.0|
|   4583468|     Desconhecido|          0.0|          0.0|          0.0|          0.0|          0.0|            0.0|
|   4583469|     Desconhecido|          0.0|          0.0|          0.0|          0.0|          0.0|            0.0|
|   4583470|    Sem problemas|        100.0|        140.0|        120.0|        140.0|        100.0|          600.0|
|   4583471|    Sem problemas|        140.0|        120.0|      

**CRIAÇÃO DA TABELA DE DIM.FATO_RESULTADOS_ENEM NA CAMADA GOLD**

In [60]:
# Criando a tabela usando Iceberg
spark.sql("""
    CREATE TABLE nessie.gold.fato_resultados_enem
    USING iceberg
    LOCATION 's3a://gold/fato_resultados_enem/'
    AS SELECT 
        ID AS ID_PARTICIPANTE,
        ID AS ID_ESCOLA,
        ID AS ID_APLICACAO_PROVA,
        ID AS ID_PROVA_OBJETIVA,
        ID AS ID_Q_SE,
        ID AS ID_REDACAO,
        NU_NOTA_CN,
        NU_NOTA_CH,
        NU_NOTA_LC,
        NU_NOTA_MT,
        TX_RESPOSTAS_CN,
        TX_RESPOSTAS_CH,
        TX_RESPOSTAS_LC,
        TX_RESPOSTAS_MT,
        TX_GABARITO_CN,
        TX_GABARITO_CH,
        TX_GABARITO_LC,
        TX_GABARITO_MT,
        NU_NOTA_COMP1,
        NU_NOTA_COMP2,
        NU_NOTA_COMP3,
        NU_NOTA_COMP4,
        NU_NOTA_COMP5,
        NU_NOTA_REDACAO
    FROM tabela_enem_silver
    LIMIT 0;
""")

# Inserindo os dados na tabela
spark.sql("""
    INSERT INTO nessie.gold.fato_resultados_enem
    SELECT 
        -- Chaves estrangeiras (FK)
        p.ID_PARTICIPANTE,
        e.ID_ESCOLA,
        ap.ID_APLICACAO_PROVA,
        pr.ID_PROVA_OBJETIVA,
        q.ID_Q_SE,
        r.ID_REDACAO,

        -- Notas e resultados das provas
        pr.NU_NOTA_CN AS NOTA_CN,
        pr.NU_NOTA_CH AS NOTA_CH,
        pr.NU_NOTA_LC AS NOTA_LC,
        pr.NU_NOTA_MT AS NOTA_MT,

        -- Respostas de múltipla escolha
        pr.TX_RESPOSTAS_CN,
        pr.TX_RESPOSTAS_CH,
        pr.TX_RESPOSTAS_LC,
        pr.TX_RESPOSTAS_MT,

        -- Gabarito das provas
        pr.TX_GABARITO_CN,
        pr.TX_GABARITO_CH,
        pr.TX_GABARITO_LC,
        pr.TX_GABARITO_MT,

        -- Notas da redação
        r.NU_NOTA_COMP1,
        r.NU_NOTA_COMP2,
        r.NU_NOTA_COMP3,
        r.NU_NOTA_COMP4,
        r.NU_NOTA_COMP5,
        r.NU_NOTA_REDACAO

    FROM
        tabela_enem_silver en
    LEFT JOIN nessie.gold.dim_participante p
        ON en.ID = p.ID_PARTICIPANTE
    LEFT JOIN nessie.gold.dim_escola e
        ON en.ID = e.ID_ESCOLA
    LEFT JOIN nessie.gold.dim_aplicacao_prova ap
        ON en.ID = ap.ID_APLICACAO_PROVA
    LEFT JOIN nessie.gold.dim_prova_objetiva pr
        ON en.ID = pr.ID_PROVA_OBJETIVA
    LEFT JOIN nessie.gold.dim_q_se q
        ON en.ID = q.ID_Q_SE
    LEFT JOIN nessie.gold.dim_redacao r
        ON en.ID = r.ID_REDACAO;
""")


DataFrame[]

In [61]:
spark.sql("""SELECT * FROM nessie.gold.fato_resultados_enem""").show()

+---------------+---------+------------------+-----------------+-------+----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+-------------+-------------+-------------+---------------+
|ID_PARTICIPANTE|ID_ESCOLA|ID_APLICACAO_PROVA|ID_PROVA_OBJETIVA|ID_Q_SE|ID_REDACAO|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|     TX_RESPOSTAS_CN|     TX_RESPOSTAS_CH|     TX_RESPOSTAS_LC|     TX_RESPOSTAS_MT|      TX_GABARITO_CN|      TX_GABARITO_CH|      TX_GABARITO_LC|      TX_GABARITO_MT|NU_NOTA_COMP1|NU_NOTA_COMP2|NU_NOTA_COMP3|NU_NOTA_COMP4|NU_NOTA_COMP5|NU_NOTA_REDACAO|
+---------------+---------+------------------+-----------------+-------+----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----

In [63]:
spark.sql("DESCRIBE FORMATTED nessie.gold.fato_resultados_enem").show()


+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|   ID_PARTICIPANTE|      int|   NULL|
|         ID_ESCOLA|      int|   NULL|
|ID_APLICACAO_PROVA|      int|   NULL|
| ID_PROVA_OBJETIVA|      int|   NULL|
|           ID_Q_SE|      int|   NULL|
|        ID_REDACAO|      int|   NULL|
|        NU_NOTA_CN|   double|   NULL|
|        NU_NOTA_CH|   double|   NULL|
|        NU_NOTA_LC|   double|   NULL|
|        NU_NOTA_MT|   double|   NULL|
|   TX_RESPOSTAS_CN|   string|   NULL|
|   TX_RESPOSTAS_CH|   string|   NULL|
|   TX_RESPOSTAS_LC|   string|   NULL|
|   TX_RESPOSTAS_MT|   string|   NULL|
|    TX_GABARITO_CN|   string|   NULL|
|    TX_GABARITO_CH|   string|   NULL|
|    TX_GABARITO_LC|   string|   NULL|
|    TX_GABARITO_MT|   string|   NULL|
|     NU_NOTA_COMP1|   double|   NULL|
|     NU_NOTA_COMP2|   double|   NULL|
+------------------+---------+-------+
only showing top 20 rows



In [65]:
spark.sql("""
    WITH medias AS (
        SELECT
            p.NU_ANO,  -- Ano do participante
            p.ID_PARTICIPANTE,  -- ID do participante
            (
                (pr.NU_NOTA_CN + pr.NU_NOTA_CH + pr.NU_NOTA_LC + pr.NU_NOTA_MT + r.NU_NOTA_REDACAO) / 5.0
            ) AS MEDIA_TOTAL  -- Média total de todas as provas (objetivas + redação)
        FROM
            nessie.gold.fato_resultados_enem f
        LEFT JOIN nessie.gold.dim_participante p
            ON f.ID_PARTICIPANTE = p.ID_PARTICIPANTE
        LEFT JOIN nessie.gold.dim_prova_objetiva pr
            ON f.ID_PROVA_OBJETIVA = pr.ID_PROVA_OBJETIVA
        LEFT JOIN nessie.gold.dim_redacao r
            ON f.ID_REDACAO = r.ID_REDACAO
    )
    SELECT
        m.NU_ANO,
        m.ID_PARTICIPANTE,
        m.MEDIA_TOTAL
    FROM
        medias m
    WHERE
        (m.NU_ANO, m.MEDIA_TOTAL) IN (
            SELECT
                NU_ANO,
                MAX(MEDIA_TOTAL) AS MEDIA_TOTAL
            FROM
                medias
            GROUP BY
                NU_ANO
        )
    ORDER BY
        m.NU_ANO, m.MEDIA_TOTAL DESC
""").show()


+------+---------------+-----------------+
|NU_ANO|ID_PARTICIPANTE|      MEDIA_TOTAL|
+------+---------------+-----------------+
|  2019|        2512768|850.8199999999999|
|  2020|       10041752|858.5799999999999|
|  2021|       11769458|           862.68|
|  2022|       15196456|855.9799999999999|
|  2023|       19660803|862.5799999999999|
+------+---------------+-----------------+



In [58]:
spark.sparkContext.cancelAllJobs()

In [ ]:
spark.stop()